# Welcome to Flow!

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import flow

In [2]:
b = flow.test_baseline()
p = flow.test_ww_param()
#f = flow.test_EP_flows()

d = flow.calc_wastewater_water_demand(data=b, target_types=p, output='l3')
for col in d.columns:
    print(col)
d

County
FIPS
State


,County,FIPS,State
0,Mendacino,2030,CA


In [3]:
d = flow.calc_collect_water_use()
for col in d.columns:
    print(col)

County
FIPS
PWD_total_total_total_total_to_RES_total_total_total_total_mgd
RES_total_total_total_total_natural_mgd
RES_total_total_total_total_public_mgd
RES_total_total_total_total_to_Con_total_total_total_total_mgd
RES_total_total_total_total_to_OD_total_total_total_total_mgd
RES_total_total_total_total_to_SD_total_total_total_total_mgd
State
WS_fresh_surfacewater_total_total_to_RES_total_total_total_total_mgd


County
FIPS
RES_total_total_total_total_natural_mgd
RES_total_total_total_total_to_Con_advanced_total_total_total_mgd
RES_total_total_total_total_to_OD_total_total_total_total_mgd
State
WS_fresh_surfacewater_total_total_to_RES_total_total_total_total_mgd


In [2]:
d = flow.calc_pws_discharge()
d.sum()

FIPS                                                                             0100101003010050100701009010110101301015010170...
State                                                                            ALALALALALALALALALALALALALALALALALALALALALALAL...
County                                                                           Autauga CountyBaldwin CountyBarbour CountyBibb...
PWS_fresh_surfacewater_total_total_to_CO_total_total_total_total_mgd                                                   3740.865749
PWS_fresh_groundwater_total_total_to_CO_total_total_total_total_mgd                                                    3380.223737
PWS_fresh_surfacewater_total_total_to_IN_total_total_total_total_mgd                                                   3072.223013
PWS_fresh_groundwater_total_total_to_IN_total_total_total_total_mgd                                                    1912.464893
PWS_saline_surfacewater_total_total_to_CO_total_total_total_total_mgd              

In [2]:

#pd.set_option('display.max_rows', None)
data = flow.configure_data()
source_parameters = flow.get_water_demand_source_parameters()
target_parameters = flow.get_water_demand_target_parameters()
x = flow.calc_water_to_nonwater_sectors(data=data, source_parameters=source_parameters, target_parameters=target_parameters, output='l5')
x.sum()

KeyError: "['fresh_groundwater_mining_mgd', 'fresh_surfacewater_mining_mgd', 'saline_groundwater_mining_mgd', 'saline_surfacewater_mining_mgd'] not in index"

In [4]:
d = flow.calc_pws_discharge()
d.sum()

FIPS                                                                             0100101003010050100701009010110101301015010170...
State                                                                            ALALALALALALALALALALALALALALALALALALALALALALAL...
County                                                                           Autauga CountyBaldwin CountyBarbour CountyBibb...
PWS_fresh_surfacewater_total_total_to_CO_total_total_total_total_mgd                                                   3740.865749
PWS_fresh_groundwater_total_total_to_CO_total_total_total_total_mgd                                                    3380.223737
PWS_fresh_surfacewater_total_total_to_IN_total_total_total_total_mgd                                                   3072.223013
PWS_fresh_groundwater_total_total_to_IN_total_total_total_total_mgd                                                    1912.464893
PWS_saline_surfacewater_total_total_to_CO_total_total_total_total_mgd              

In [9]:
d = flow.get_water_use_2015()
d['PS-WSWSa'].sum()

7.210000000000001

In [7]:
df = flow.prep_electricity_demand_data()
demand_columns = df.columns[1:5].to_list()
demand_columns
for d in demand_columns:
    df[d] = df[d] * df['pop_weight']
    df[d] = df[d].round(4)
df.sum()

State                                                             AKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAKAK...
EG_total_total_total_total_to_RES_total_total_total_total_bbtu                                         3235940.6618
EG_total_total_total_total_to_CO_total_total_total_total_bbtu                                          2859809.8199
EG_total_total_total_total_to_IN_total_total_total_total_bbtu                                          2004424.2834
EG_total_total_total_total_to_TR_total_total_total_total_bbtu                                             5451.1193
FIPS                                                              0201302016020200205002060020680207002090021000...
County                                                            Aleutians East BoroughAleutians West Census Ar...
pop_weight                                                                                                     53.0
dtype: object

In [8]:
d = flow.prep_electricity_demand_data()
d.dtypes

State                                                              object
EG_total_total_total_total_to_RES_total_total_total_total_bbtu    float64
EG_total_total_total_total_to_CO_total_total_total_total_bbtu     float64
EG_total_total_total_total_to_IN_total_total_total_total_bbtu     float64
EG_total_total_total_total_to_TR_total_total_total_total_bbtu     float64
FIPS                                                               object
County                                                             object
dtype: object

In [5]:
df = flow.get_state_electricity_demand_data()

    # build renaming dictionary
rename_dict = {"RESIDENTIAL": "EG_total_total_total_total_to_RES_total_total_total_total_bbtu",
                   "COMMERCIAL": "EG_total_total_total_total_to_CO_total_total_total_total_bbtu",
                   "INDUSTRIAL": "EG_total_total_total_total_to_IN_total_total_total_total_bbtu",
                   "TRANSPORTATION": "EG_total_total_total_total_to_TR_total_total_total_total_bbtu"}

    # prep dataframe
df = df.dropna(subset=["Month"])  # drop rows where month is blank
df = df.dropna(subset=["Ownership"])  # Drop state totals and state adjustments
df = df[df.Ownership != "Behind the Meter"]  # removing behind the meter generation
df = df.groupby("State", as_index=False).sum()  # get total by state
df = df[["State", "RESIDENTIAL", "COMMERCIAL", "INDUSTRIAL", "TRANSPORTATION"]]

    # convert electricity demand values from mwh to bbtu
column_list = df.columns[1:]
for col in column_list:
    df[col] = df[col].apply(flow.convert_mwh_bbtu)

    # rename columns to add descriptive language
df.rename(columns=rename_dict, inplace=True)
    

    # split out into county values based on percent of state population
df = flow.calc_population_county_weight(df)
demand_columns = df.columns[1:5].to_list()
for d in demand_columns:
    df[d] = df[d] * df['pop_weight']
    df[d] = df[d].round(4)
df

,State,EG_total_total_total_total_to_RES_total_total_total_total_bbtu,EG_total_total_total_total_to_CO_total_total_total_total_bbtu,EG_total_total_total_total_to_IN_total_total_total_total_bbtu,EG_total_total_total_total_to_TR_total_total_total_total_bbtu,FIPS,County,pop_weight
0,AK,28.6849,37.6439,18.9169,0.0,02013,Aleutians East Borough,0.004524
1,AK,48.9558,64.2459,32.2850,0.0,02016,Aleutians West Census Area,0.007722
2,AK,2564.5103,3365.4743,1691.2244,0.0,02020,Municipality of Anchorage,0.404499
3,AK,154.0793,202.2023,101.6111,0.0,02050,Bethel Census Area,0.024303
4,AK,7.6585,10.0504,5.0505,0.0,02060,Bristol Bay Borough,0.001208
...,...,...,...,...,...,...,...,...
3137,WY,608.3505,920.4065,2584.4084,0.0,56037,Sweetwater County,0.076140
3138,WY,315.2446,476.9507,1339.2292,0.0,56039,Teton County,0.039455
3139,WY,283.8497,429.4515,1205.8565,0.0,56041,Uinta County,0.035526
3140,WY,113.5290,171.7641,482.2963,0.0,56043,Washakie County,0.014209


In [5]:
y = flow.construct_nested_dictionary(flow.get_fuel_demand_target_parameters())
#len(flow.get_fuel_demand_source_parameters())
#x = flow.get_fuel_demand_source_parameters()
y

{'CO': {'total': {'total': {'total': {'total': {nan: nan}}}}},
 'EG': {'biomass': {'combinedcycle': {'nocooling': {'total': {nan: nan}},
    'oncethrough': {'total': {nan: nan}},
    'tower': {'total': {nan: nan}}},
   'combustionturbine': {'nocooling': {'total': {nan: nan}},
    'tower': {'total': {nan: nan}}},
   'fuelcell': {'nocooling': {'total': {nan: nan}}},
   'internalcombustion': {'nocooling': {'total': {nan: nan}},
    'tower': {'total': {nan: nan}}},
   'steam': {'complex': {'total': {nan: nan}},
    'nocooling': {'total': {nan: nan}},
    'oncethrough': {'total': {nan: nan}},
    'pond': {'total': {nan: nan}},
    'tower': {'total': {nan: nan}}}},
  'coal': {'combinedcycle': {'nocooling': {'total': {nan: nan}}},
   'steam': {'complex': {'total': {nan: nan}},
    'nocooling': {'total': {nan: nan}},
    'oncethrough': {'total': {nan: nan}},
    'pond': {'total': {nan: nan}},
    'tower': {'total': {nan: nan}}}},
  'geothermal': {'binarycycle': {'nocooling': {'total': {nan: na

In [6]:
x = flow.prep_fuel_demand_data()
x

,FIPS,State,County,EC_coal_total_total_total_to_CO_total_total_total_total_bbtu,EC_coal_total_total_total_to_IN_total_total_total_total_bbtu,EC_biomass_total_total_total_to_TR_total_total_total_total_bbtu,EC_geothermal_total_total_total_to_CO_total_total_total_total_bbtu,EC_geothermal_total_total_total_to_RES_total_total_total_total_bbtu,EC_natgas_total_total_total_to_TR_total_total_total_total_bbtu,EC_natgas_total_total_total_to_CO_total_total_total_total_bbtu,...,EC_natgas_total_total_total_to_RES_total_total_total_total_bbtu,EC_petroleum_total_total_total_to_TR_total_total_total_total_bbtu,EC_petroleum_total_total_total_to_CO_total_total_total_total_bbtu,EC_petroleum_total_total_total_to_IN_total_total_total_total_bbtu,EC_petroleum_total_total_total_to_RES_total_total_total_total_bbtu,EC_solar_total_total_total_to_CO_total_total_total_total_bbtu,EC_solar_total_total_total_to_RES_total_total_total_total_btu,EC_biomass_total_total_total_to_RES_total_total_total_total_bbtu,EC_biomass_total_total_total_to_CO_total_total_total_total_bbtu,EC_wind_total_total_total_to_CO_total_total_total_total_bbtu
0,01001,AL,Autauga County,0.0,9.014449,2.910620,0.0,0.012845,2.907506,3.354745,...,4.366384,60.343235,1.455115,8.262693,0.717891,0.003244,0.009861,0.275453,0.040351,20.486311
1,01003,AL,Baldwin County,0.0,122.115684,39.429180,0.0,0.174007,39.386997,45.445588,...,59.149922,817.449354,19.711954,111.931907,9.725032,0.043941,0.133581,3.731474,0.546627,277.521105
2,01005,AL,Barbour County,0.0,2.064820,0.666697,0.0,0.002942,0.665984,0.768427,...,1.000149,13.822021,0.333304,1.892625,0.164438,0.000743,0.002259,0.063094,0.009243,4.692526
3,01007,AL,Bibb County,0.0,1.500770,0.484575,0.0,0.002138,0.484056,0.558515,...,0.726937,10.046243,0.242255,1.375614,0.119518,0.000540,0.001642,0.045859,0.006718,3.410663
4,01009,AL,Blount County,0.0,9.788048,3.160403,0.0,0.013947,3.157021,3.642641,...,4.741097,65.521753,1.579990,8.971779,0.779499,0.003522,0.010707,0.299092,0.043814,22.244399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3218,72151,PR,Yabucoa Municipio,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3219,72153,PR,Yauco Municipio,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3220,78010,VI,St. Croix County,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3221,78020,VI,St. John County,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
l1_df = pd.DataFrame.from_dict(x, orient='index').transpose()


In [20]:

df = pd.DataFrame()
for i in range(len(data)):
    df = df.append({'A': 0}, ignore_index=True)
df

,A
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3219,0.0
3220,0.0
3221,0.0
3222,0.0


In [2]:
for col in x.columns:
    print(col)

FIPS
State
County
EC_biomass_total_total_total_to_EG_biomass_combinedcycle_nocooling_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_combinedcycle_oncethrough_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_combinedcycle_tower_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_combustionturbine_nocooling_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_combustionturbine_tower_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_fuelcell_nocooling_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_internalcombustion_nocooling_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_internalcombustion_tower_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_steam_complex_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_steam_nocooling_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_steam_oncethrough_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_steam_pond_total_bbtu
EC_biomass_total_total_total_to_EG_biomass_steam_tower_total_bbtu
EC_coal_total_

In [2]:
df.sum()

FIPS                                    0100101003010050100701009010110101301015010170...
State                                   ALALALALALALALALALALALALALALALALALALALALALALAL...
County                                  Autauga CountyBaldwin CountyBarbour CountyBibb...
ec_consumption_to_eg_generation_bbtu                                          1776580.364
eg_generation_to_re_bbtu                                                   1126114.391136
eg_generation_to_es_bbtu                                                    650465.972864
dtype: object

In [4]:
fail_parameter_data = flow.get_electricity_generation_efficiency_parameters()
fail_parameter_data = fail_parameter_data.drop(fail_parameter_data.columns[2], axis=1)
fail_parameter_data

,fuel_type,sub-fuel,value
0,biomass,total,0.3
1,coal,total,0.3
2,geothermal,total,0.3
3,hydro,total,0.3
4,natgas,total,0.3
5,nuclear,total,0.3
6,oil,total,0.3
7,other,total,0.3
8,solar,total,0.3
9,wind,total,0.3


In [ ]:
parameters = flow.get_electricity_generation_efficiency_parameters()
test_dict = flow.construct_nested_dictionary(parameters)
test_fuel_type = list(test_dict.keys())[0]
test_sub_fuel_type = list(test_dict[test_fuel_type].keys())[0]

test_sub_fuel_type

In [ ]:
#df = flow.calc_electricity_energy_discharge(data=d, total=True)
df['eg_generation_biomass_to_es_bbtu'][4]

In [ ]:
df.sum()

In [ ]:
df = flow.get_electricity_generation_efficiency_parameters()
x = flow.construct_nested_dictionary(df)

x['biomass']['total']['efficiency']

In [ ]:
efficiency_dict = {'biomass':{'total'}, 'coal':{'total'}, 'geothermal':{'total'}, 'hydro':{'total'}, 'natgas':{'total'}, 'nuclear':{'total'},
                       'oil':{'total'}, 'other':{'total'}, 'solar':{'total'}, 'wind':{'total'}}
efficiency_dict

In [ ]:
d = df.groupby(group1).apply(lambda x: dict(zip(x[parameter])))

In [ ]:
group1 = df.columns[0]
group2 = df.columns[1]
group3 = df.columns[2]
group4 = df.columns[3]
parameter = df.columns[-2]
value = df.columns[-1]


d = df.groupby(group1).apply(lambda x: dict(zip(x[parameter], x[value])))
d = d.to_dict()
d


In [ ]:
d = df.groupby(group1).apply(lambda a: dict(a.groupby(group2).apply(lambda b: dict(b.groupby(group3).apply(
            lambda b: dict(b.groupby(group4).apply(lambda x: dict(zip(x[parameter], x[value])))))))))

In [ ]:
main_grouper = df.columns[0]
nest_list = df.columns[1:].to_list()
second_list =  df.columns[2:]
sub_nest_grouper = nest_list[0]

In [ ]:
fuel_dict = df.groupby(main_grouper)[nest_list].apply(lambda x: x.set_index([nest_list]).to_dict(orient='index')).to_dict()
fuel_dict

In [ ]:
for item in res:
    print(res['biomass'])
    

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_list = data.select_dtypes(include=numerics).columns.tolist()
record_list = []
for column in numeric_list:
    x = data.columns.get_loc(column)
    record_list.append(x)
length = min(record_list)


main_grouper = data.columns[0]
nest_list = data.columns[1:].to_list()
sub_nest_grouper = nest_list[0]
fuel_dict = data.groupby(main_grouper)[nest_list].apply(lambda x: x.set_index(sub_nest_grouper).to_dict(orient='index')).to_dict()
fuel_dict

In [ ]:
main_grouper = data.columns[1]
nest_list = data.columns[1:].to_list()
sub_nest_grouper = nest_list[0]

    # no sub-fuel types
main_type_unique = data[main_grouper].is_unique
sub_type_unique = data[sub_nest_grouper].is_unique

if main_type_unique and sub_type_unique:
    fuel_dict = data.set_index(main_grouper).to_dict(orient='index')
elif not main_type_unique and not sub_type_unique:

    fuel_dict = data.groupby(main_grouper)[nest_list].apply(lambda x: x.set_index(sub_nest_grouper).to_dict(orient='index')).to_dict()
else:
    pass

fuel_dict

In [ ]:
#data.apply(lambda x: x.set_index('fuel_types').to_dict(orient='index'))
nest_list = data.columns[2:].to_list()
fuel_dict = data.groupby('fuel_type')#[nest_list].apply(lambda x: x.set_index('fuel_type').to_dict(orient='index')).to_dict()
fuel_dict = data.set_index('fuel_type').to_dict(orient='index')
fuel_dict

In [ ]:
data.head()
d = flow.construct_dictionary(data)
d

In [ ]:
df.columns[1]
nest_list = df.columns[2:].to_list()
nest_list[0]

In [ ]:
my_dict = {1:{'name':'x', 'age':24}, 2:{'name':'y', 'age':21}, 3:{'name':'z', 'age':25}}

#my_dict
#def count(d):
#    return max(count(v) if isinstance(v,dict) else 0 for v in d.values()) + 1

#print(count(my_dict))

In [ ]:
    nested_dict = df.groupby(main_grouper)[nest_list].apply(lambda x: x.set_index(sub_nest_grouper).to_dict(orient='index')).to_dict()


In [ ]:
import flow

In [ ]:
x = fresh_groundwater_coal_generation_mgd + fresh_surfacewater_coal_generation_mgd + fresh_wastewater_coal_generation_mgd + saline_groundwater_coal_generation_mgd + saline_surfacewater_coal_generation_mgd
x
z = coal_generation_surface_discharge_mgd + coal_generation_ocean_discharge_mgd
x - coal_generation_consumption_mgd 

In [ ]:
z

In [ ]:
df = df.groupby('fuel_type')[['sub_fuel','fuel_efficiency','parameter2']].apply(lambda x: x.set_index('sub_fuel').to_dict(orient='index')).to_dict()
df

In [ ]:
round(df).sum()

In [ ]:
54+45

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
48384*0.00016149393844372542

In [ ]:
df.sum()

In [ ]:

df['total_water'] = df['fresh_surfacewater_natgas_unconventional_mgd'] + df['fresh_groundwater_natgas_unconventional_mgd']
df['intensity'] = np.where(df['natgas_unconventional_production_bbtu']>0, df['total_water']/(df['natgas_unconventional_production_bbtu']/365), 0)
df['sw_pct'] = np.where( df['total_water']>0, df['fresh_surfacewater_natgas_unconventional_mgd']/df['total_water'], 0)
df['gw_pct'] = np.where( df['total_water']>0, df['fresh_groundwater_natgas_unconventional_mgd']/df['total_water'], 0)

df.groupby("State", as_index=False).sum()
x = df['intensity'].mean()
x

In [ ]:

# initialize data of lists.
d_list = {'Name':['Tom', 'Tom','nick', 'krish', 'jack'],
        'Age':[20, 22, 21, 19, 18],
         'height':[20,21, 21, 19, 18]}
 
# Create DataFrame
d_list = pd.DataFrame(d_list)
d_list.head()

d_list = d_list.groupby('Name').apply(lambda x: x.set_index('Age').to_dict(orient='index')).to_dict()
d_list

In [ ]:
df3 = pd.DataFrame([
           ['A', 123, 1], 
           ['B', 345, 5], 
           ['C', 712, 4],
           ['B', 768, 2], 
           ['B', 768, 3], 
           ['A', 123, 9], 
           ['C', 178, 6], 
           ['C', 178, 5],  
           ['A', 321, 3]])

d = {k: f.groupby('subgroup')['selectedCol'].apply(list).to_dict()
     for k, f in df.groupby('maingroup')}
d

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
df = data.copy()

In [ ]:
# def calc_coal_production_water_use(data: pd.DataFrame, mine_types = None, water_types=None, discharge_types = None,
#                                   regions=3, total=False):
#    """calculates total water use in hydroelectric generation by region.
#
#        Water use is determined by applying hydropower water intensity rates (mg/bbtu) to daily hydropower generation
#        (bbtu) to get mgd in each region. Water use can be interpreted as the amount of water that passes through
#        the hydropower facility on a daily basis, based on average power generation per day. If region-level hydropower
#        water intensity rates are not available, an intensity assumption is applied to all hydropower generation.
#        Surface discharge from hydropower is also calculated. This is assumed to be equal to water use in hydropower.
#        No water is assumed to be consumed or evaporated given that the water included in this calculation only
#        includes water that instantaneously passes through the hydropower facility and is immediately discharged back
#        to the surface water source.
#
#        :param data:                        DataFrame of input data
#        :type data:                         DataFrame
#
#        :param hydro_water_intensity:       The assumed million gallons required to get a billion btu from hydropower
#        :type hydro_water_intensity:        flt
#
#        :param regions:                     gives the number of columns in the dataset that should be treated as region
#                                            identifiers (e.g. "Country", "State"). Reads from the first column in the
#                                            dataframe onwards.
#        :type regions:                      int
#
#        :return:                            DataFrame of total water use by hydropower generation (mgd) and hydropower
#                                            discharge to surface waters (mgd) by region.
#
#        """
#
#    if mine_types is None:
#        mine_type_dict = {'surface': 0.000007, 'ground': 0.000029}
#    else:
#        mine_type_dict = mine_types
#
#    if water_types is None:
#        water_type_dict = {'fresh': {'surfacewater': {'flow_fraction': .5, 'consumption_fraction': .5},
#                                    'groundwater': {'flow_fraction': .5, 'consumption_fraction': .5}},
#                           'saline': {'surfacewater': {'flow_fraction': .5, 'consumption_fraction': .5},
#                                    'groundwater': {'flow_fraction': .5, 'consumption_fraction': .5}}}
#    else:
#        water_type_dict = water_types
#
#    if discharge_types is None:
#        discharge_type_dict = {'surface': 1}
#    else:
#        discharge_type_dict = discharge_types
#
#    # load data
#    df = data
#
#    region_list = df.columns[:regions].tolist()
#    output_df = df[region_list].copy()
#    total_df = df[region_list].copy()
#
#    for water_type in water_type_dict:
#        for water_source in water_type_dict[water_type]:
#            coal_water_total_name = f'{water_type}_{water_source}_coal_mgd'
#            output_df[coal_water_total_name] = 0
#            total_df[coal_water_total_name] = 0
#
#    # calculate total water to all mining (inclusive of coal)
#    df['total_water_mining_mgd'] = 0
#    for water_type in water_type_dict:
#        for water_source in water_type_dict[water_type]:
#            mining_water_flow_name = f'{water_type}_{water_source}_mining_mgd'
#            if mining_water_flow_name in df.columns:
#                df['total_water_mining_mgd'] = df['total_water_mining_mgd'] + df[mining_water_flow_name]
#            else:
#                pass
#
#    # calculate percent of total mining flows by water type and water source
#    for water_type in water_type_dict:
#        for water_source in water_type_dict[water_type]:
#
#            mining_water_flow_pct_name = f'{water_type}_{water_source}_mining_pct'
#
#            if mining_water_flow_pct_name in df.columns: #if the percent of water flows to coal mining in data
#                pass
#
#            else:  # if only water flows to all mining in data
#                mining_water_flow_name = f'{water_type}_{water_source}_mining_mgd'
#                if mining_water_flow_name in df.columns:  # if water type and water source to all mining in data
#                    df[mining_water_flow_pct_name] = np.where(df['total_water_mining_mgd'] > 0,
#                                                          (df[mining_water_flow_name] / df['total_water_mining_mgd']),
#                                                          0)
#                else:  # if no breakdown of water source and water type to all mining in data
#                    df[mining_water_flow_pct_name] = water_type_dict[water_type][water_source]['flow_fraction']
#
#            # calculate water in coal mining for each mine type by water type and water source
#            for mine_type in mine_type_dict:
#                coal_shortton_name = f'{mine_type}_coal_shortton'
#                if coal_shortton_name in df.columns:
#                    coal_water_intensity_name = f'{mine_type}_coal_water_intensity_mg_per_shortton'
#                    coal_water_total_name = f'{water_type}_{water_source}_coal_mgd'
#                    if coal_water_intensity_name in df.columns:  # if regional water intensity values available
#                        coal_water_mgd_name = f"{water_type}_{water_source}_{mine_type}_coal_mgd"
#                        output_df[coal_water_mgd_name] = df[coal_water_intensity_name] \
#                                                         * (df[coal_shortton_name]/365) \
#                                                         * df[mining_water_flow_pct_name]
#                        output_df[coal_water_total_name] = output_df[coal_water_total_name] \
#                                                           + output_df[coal_water_mgd_name]
#                        total_df[coal_water_total_name] = total_df[coal_water_total_name] \
#                                                          + output_df[coal_water_mgd_name]
#
#                    else:  # if no regional water intensity available, use assumption
#                        coal_water_mgd_name = f"{water_type}_{water_source}_{mine_type}_coal_mgd"
#                        output_df[coal_water_mgd_name] = mine_type_dict[mine_type] \
#                                                         * (df[coal_shortton_name]/365) \
#                                                         * df[mining_water_flow_pct_name]
#                        output_df[coal_water_total_name] = output_df[coal_water_total_name] \
#                                                           + output_df[coal_water_mgd_name]
#                        total_df[coal_water_total_name] = total_df[coal_water_total_name] \
#                                                          + output_df[coal_water_mgd_name]
#                else:  # if no coal mining available
#                    pass
#
## TODO pick things up here
#
#            # calculate consumption of water in coal mining by water type and water source
#
#            # look for region- mining consumption fraction, apply it
#
#            # otherwise use dictionary values water_type_dict[water_type][water_source]['consumption_fraction']
#
#            ## look for region- mining discharge fraction, apply it
#            # for discharge_type in discharge_type dictionary:
#                #discharge_type_name = f'{water_type}_{water_source}_{mine_type}_{discharge_type}_discharge_mgd
#                #discharge_pct_name = f'{water_type}_{water_source}_{mine_type}_{discharge_type}_discharge_pct
#
#                # if discharge_pct_name in df.columns:
#                    # apply pct to total discharge
#                # else:
#                    # apply pct from dictionary
#            #
#
#    # TODO add in consumption and surface discharge


#    if total:
#        df = total_df
#    else:
#        df = output_df
#
#    return df

In [ ]:
for mine_type in fuel_water_type_dict['coal']:
    for water_type in fuel_water_type_dict['coal'][mine_type]:
        print(water_type)

In [ ]:
df.sum()

In [ ]:
df = flow.prep_electricity_generation()
df

In [ ]:
data = flow.configure_data()
sector_types = {'crop_irrigation': .15, 'golf_irrigation': .10}


df=flow.calc_water_conveyance_losses(data=data, sector_types=sector_types, total=True)
for col in df.columns:
    print(col)

In [ ]:
df = df.groupby('State', as_index=False).sum()
df

In [ ]:
data = flow.configure_data()
df = flow.calc_energy_agriculture(data=data, total=True)
for col in df.columns:
    print(col)

In [ ]:
df = flow.calc_wastewater_exports(data=data)
for col in df.columns:
    print(col)

In [2]:
x.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\fuel_types.csv')

In [ ]:
name_list = ['electricity to rejected energy', 'energy discharge from sectoral use', 'energy in wastewater',
                      'energy in agriculture', 'energy in public water supply', 'energy production exports',
                     'water discharge from sectoral use']
name_list.index('electricity to rejected energy')

In [ ]:
water_discharge_dict = {'surfacewater': {'fresh': {'wastewater': 0, 'ocean': 0, 'surface': 1},
                                        'saline': {'wastewater': 0, 'ocean': 1, 'surface': 0}},
                        'groundwater': {'fresh': {'wastewater': 0, 'ocean': 0, 'surface': 1},
                                        'saline': {'wastewater': 0, 'ocean': 1, 'surface': 0}},
                                'pws': {'fresh': {'wastewater': 1, 'ocean': 0, 'surface': 0}}
                             }

In [ ]:
sector_consumption_dict = {'residential': {'groundwater': {'saline': 0, 'fresh': .3},
                                                   'surfacewater': {'saline': 0, 'fresh': .3},
                                                   'pws': {'fresh': .3}},
                            'commercial': {'groundwater': {'saline': 0, 'fresh': .15},
                                           'surfacewater': {'saline': 0, 'fresh': .15},
                                           'pws': {'fresh': .15}},
                            'industrial':{'groundwater': {'saline': .003, 'fresh': .15},
                                          'surfacewater': {'saline': .003, 'fresh': .15},
                                          'pws': {'fresh': .15}},
                            'mining': {'groundwater': {'saline': .03, 'fresh': .15},
                                       'surfacewater': {'saline': .03, 'fresh': .15}},
                            'crop_irrigation': {'groundwater': {'saline': 0, 'fresh': .3},
                                                'surfacewater': {'saline': 0, 'fresh': .3}},
                            'livestock':  {'groundwater': {'saline': 0, 'fresh': .87},
                                           'surfacewater': {'saline': 0, 'fresh': .87}},
                            'aquaculture': {'groundwater': {'saline': 0, 'fresh': .5},
                                            'surfacewater': {'saline': 0, 'fresh': .5}}}

In [ ]:
df = data.copy()

In [ ]:
saline_groundwater_industrial_mgd                                                                   42.9
saline_surfacewater_industrial_mgd                                                                742.92

In [ ]:
df = flow.calc_sectoral_use_water_discharge(data=data, total=True)

In [ ]:
x=16987.779934 +  6225.87913 + 4263.081345
x

In [ ]:
df1=flow.calc_sectoral_use_water_discharge(data=data, total=True)
df1 = df1[['FIPS', 'residential_wastewater_discharge_mgd', 'commercial_wastewater_discharge_mgd', 'industrial_wastewater_discharge_mgd']]

In [ ]:
df2 = flow.prep_wastewater_data()
df2 = df2[['FIPS', 'State', 'County', 'municipal_wastewater_mgd', 'infiltration_wastewater_mgd']]

In [ ]:
df3 = pd.merge(df2, df1, how='left')
df3['sector_total'] = (df3['residential_wastewater_discharge_mgd'] 
                                                 + df3['commercial_wastewater_discharge_mgd'] 
                                                 + df3['industrial_wastewater_discharge_mgd'])
df3['diff'] = df3['municipal_wastewater_mgd'] - df3['sector_total']
df3.sum()

In [ ]:
df = pd.DataFrame()
df_sector_discharge = flow.calc_sectoral_use_water_discharge(data=data, total=True)
df_sector_discharge = df_sector_discharge[['FIPS', 'State', 'County',
                                              'residential_wastewater_discharge_mgd',
                                              'commercial_wastewater_discharge_mgd',
                                              'industrial_wastewater_discharge_mgd']]



df['total_wastewater_discharge'] = df_sector_discharge[['residential_wastewater_discharge_mgd',
                                              'commercial_wastewater_discharge_mgd',
                                              'industrial_wastewater_discharge_mgd']].sum(axis=1)

df['wastewater_exports_mgd'] = data['municipal_wastewater_mgd'] - df['total_wastewater_discharge']
df

In [ ]:
df1 = flow.calc_electricity_rejected_energy(data=data)
df2 = flow.calc_sectoral_use_energy_discharge(data=data)
df3 = flow.calc_energy_wastewater(data=data)
df4 = flow.calc_energy_agriculture(data=data)
df5 = flow.calc_energy_pws(data=data)
df6 = flow.calc_energy_production_exports(data=data)
df7 = flow.calc_sectoral_use_water_discharge(data=data)
df_list = [df1, df2, df3, df4, df5, df6, df7]
df_list

In [ ]:
df = flow.calc_sectoral_use_water_discharge(data=data)
for col in df.columns:
    print(col)

In [ ]:
df2 = df[['residential_saline_groundwater_consumption_mgd',
'residential_fresh_groundwater_consumption_mgd',
'residential_saline_surfacewater_consumption_mgd',
'residential_fresh_surfacewater_consumption_mgd',
'residential_fresh_pws_consumption_mgd',]].sum()
#df[['residential_fresh_pws_consumption_mgd']].sum()
df2

In [ ]:
x = 733+17+6309
x

In [ ]:
df = flow.calc_sectoral_use_water_discharge(data=data, total=True)
df.sum()

In [ ]:
df = flow.prep_consumption_fraction()
df.sum()

In [ ]:
df = flow.prep_water_use_2015(variables=['County','fresh_surfacewater_residential_mgd', 
                                         'fresh_groundwater_residential_mgd', 'fresh_pws_residential_mgd']).sum(axis=0)
df

In [ ]:
df = flow.calc_energy_wastewater(data=data, total=True)
#df = df[df.State == "TX"]
for col in df.columns:
    print(col)

In [ ]:
total_df

In [ ]:
#test_dict.update({'test_zero':x})
test_dict
rejected_energy_total = pd.DataFrame.from_dict(test_dict, orient='index').transpose().sum(axis=1)
rejected_energy_total



In [ ]:
x = np.zeros((len(df), 1))
test_dict.update({'test_zero': x})
test_dict['test_zero']

In [ ]:
# Move this to top

    # calculate rejected energy and energy services for each fuel, agriculture, water, and pumping type
    for water_type in water_types_list:
        for pumping_type in pumping_type_dict:
            for agriculture_type in agriculture_type_list:
                for fuel_type in fuel_type_dict:
                    fuel_type_efficiency = 'agriculture_' + fuel_type + "_efficiency_pct"
                    if f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu' in energy_value_dict:
                        rejected_energy_name = f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_rejected_energy_bbtu'
                        energy_services_name = f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_energy_services_bbtu'
                        if fuel_type_efficiency in df.columns:
                            #df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_rejected_energy_bbtu'] = (1 - df[fuel_type_efficiency]) \
                            #                                                                                         * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']
                            #df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_energy_services_bbtu'] = (df[fuel_type_efficiency]) \
                            #                                                                                         * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']

                            rejected_energy_value = (1 - df[fuel_type_efficiency]) * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']
                            energy_services_value = (df[fuel_type_efficiency]) * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']

                            rejected_energy_dict.update({rejected_energy_name: rejected_energy_value})
                            energy_services_dict.update({energy_services_name: energy_services_value})

                        # if there's no fuel efficiency column, use defaults
                        else:
                            #df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_rejected_energy_bbtu'] = (1 -fuel_type_dict[fuel_type]) \
                            #                                                                             * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']
                            #df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_energy_services_bbtu'] = (fuel_type_dict[fuel_type]) \
                            #                                                                             * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']

                            rejected_energy_value = (1 -fuel_type_dict[fuel_type]) * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']
                            energy_services_value = (fuel_type_dict[fuel_type]) * df[f'{fuel_type}_{agriculture_type}_{water_type}_{pumping_type}_bbtu']

                            rejected_energy_dict.update({rejected_energy_name: rejected_energy_value})
                            energy_services_dict.update({energy_services_name: energy_services_value})

                        #df[f'{agriculture_type}_total_rejected_energy_bbtu'] = df[f'{agriculture_type}_total_rejected_energy_bbtu'] + df[f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_rejected_energy_bbtu']
                        #df[f'{agriculture_type}_total_energy_services_bbtu'] = df[f'{agriculture_type}_total_energy_services_bbtu'] + df[ f'{agriculture_type}_{water_type}_{pumping_type}_{fuel_type}_energy_services_bbtu']
                    else:
                        pass

In [ ]:
df= df[df.State == "CA"]
df = df['electricity_irrigation_ibt_rejected_energy_bbtu']
df.sum()

In [ ]:
df = flow.calc_energy_agriculture(data=data, total=True)
df

In [ ]:
test_dict = {"Gfg" : [4, 7, 5]}
  
# printing original dictionary
print("The original dictionary is : " + str(test_dict))
  
# initializing K 
K = 1
  
# one liner, values() getting all value according to keys
res = [sub[K] for sub in test_dict.values()]
res

In [ ]:
df = flow.calc_energy_agriculture(data=data, total=True)
df
#df.groupby("State", as_index=False).sum()

In [ ]:
df = flow.prep_pumping_intensity_data()
df

In [ ]:
df = flow.prep_irrigation_fuel_data()
df

In [ ]:
df = flow.calc_electricity_wastewater(data=data)
df

In [ ]:
df = flow.calc_sectoral_use_energy_discharge(data=data, total=True)
df

In [ ]:
data.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\run4.csv')

To override the data inputs, flow.configure_data() can be run.
If run with no arguments provided, this function aggregates all baseline data to calculate flow values for the united states in 2015. Parameters are set, however, such that this function can be used to swap out datafiles. 

For example, if alternative US county level data was available for interbasin transfer electricity use, that could be used instead of the default data.

In [ ]:
df = flow.configure_data()
df

In [ ]:
is_NaN = data.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = data[row_has_NaN]
rows_with_NaN

In [ ]:
df2.to_csv(r'C:\Users\mong275\Local Files\Sankey_Notebook_Development\SankeyFiles\run2.csv')

<h1> Public Water Supply </h1>

## Water in Public Water Supply

### Public Water Supply Withdrawals

The USGS 2015 dataset provides public water supply withdrawals directly under data codes 'PS-WGWFr', 'PS-WSWFr','PS-WGWSa', 'PS-WSWSa' for each county.

In [ ]:
df = flow.prep_water_use_2015(variables=['State','County','PS-WGWFr', 'PS-WSWFr','PS-WGWSa', 'PS-WSWSa'])
df.head()               

### Public Water Supply Deliveries to Sectors, Exports, and Imports

The USGS 2015 dataset provide public water supply data in three categories: (1) total public water supply withdrawals, (2) public water supply deliveries to domestic end users, and (3) public water supply deliveries to thermoelectric cooling.

The 1995 USGS dataset provides public water supply deliveries to domestic, industrial, commercial, thermoelectric cooling, and losses/exports separately. To estimate the deliveries to commercial and industrial customers in 2015, the ratio of each to the sum of deliveries to residential and thermoelectric cooling is calculated from the 1995 data. These ratios are applied to the 2015 residential and thermoelectric cooling public water supply delivery values. 

Additionally, public water suppy exports and imports are calculated. When the sum of water deliveries is greater than the water withdrawn for public water supply, the difference is considered to be imported. When water withdrawn is greater, excess water after deliveries is considered to be exported.

Caveats and assumptions:
* This methodology assumes that if no public water supply was delivered to residential or thermoelectric cooling in 1995, then deliveries to industrial and commercial sectors would likewise be zero.

Potential mods
 for import pws counties, take 15% of withdrawals and imports as losses and add it on to the top of imports
 for export pws counties, take 15% of withdrawals as losses, take it out of exports, any remaining put as imports

In [ ]:
df = flow.calc_pws_discharge()
df.head()

## Energy in Public Water Supply

Energy in public water supply includes
groundwater pumping, surface water pumping, desalination (when applicable), surface water treatment, groundwater treatment, distribution, and interbasin transfers (when appropriate).

An assumption is made that water that goes through desalination additionally requires surface water or groundwater treatment

The same groundwater and surface water pumping intensities are used from agriculture sector.

Note that interbasin transfer data that is provided to this function is already in bbtu, not in water flows. 
For non-ibt data, the option is provided for the user to use region-level energy intensity values (as in groundwater and surface water pumping). When values are not available, assumptions for each in kWh/MG are set as default values that can be overriden.

intensity values are used from tidwell and LLNL

In [ ]:
df = flow.calc_electricity_public_water_supply(total=True)
df.head()

# Wastewater Treatment
Wastewater treatment facility data is provided by EPA for the year 2012. These values are assumed for 2015. Values are provided on a treatment plant level with data on the following variables: 
Flow:
    * Total water flows (MGD) treated by the facility
    * Total infiltration flows treated by the facility (leaks into pipes and conduits)
Treatment:
    * Treatment type of each treatment plant (primary, secondary, advanced)
Discharge:
    * discharge location of each plant, broken down by percent of flow to each discharge location (e.g., ocean discharge, suface discharge, irrigation reuse)
Location:
    * county information is included for each power plant
    
This data allows us to determine total water flows processed and discharged by wastewater facilities at a county level by treatment type. The treatment type allows us to calculate the electricity use by county for wastewater treatment,

## Water in Wastewater Treatment

### Water Inflows to Wastewater Treatment Facilities
Total flows processed by wastewater treatment facilities are provided in the EPA dataset directly. However, this data will have discrepencies from the water data delivered to various sectors. To estimate water flows from residential, commercial, and industrial sectors to wastewater treatment facilities, an assumption is made that all water delivered from public water supply to these sectors is sent to wastewater treatment facilities. This assumption is also used to fill wastewater treatment flow values for regions that are unavailable in the EPA dataset. For example, south carolina does not have wastewater treatment facility values in the EPA dataset. It is assumed, therefore, that total wastewater treatment flows in south carolina and equal to the total public water supply flows for each county and is attributed to municipal discharge rather than infiltration. It is assumed that 80% of treatment flows are advanced treatment and 20% are secondary treatment. It is also assumed that all discharge is sent to surface. This follows average values for the state of North Carolina which were used as a proxy in this instance.

### Quantity of Wastewater Processed by Treatment Type
Wastewater treatment plant treatment type is provided in the EPA data directly. These values are binned for simplification into no treatment, primary, secondary, and advanced treatment. When no treatment type was provided for a facility, it was assumed that they were secondary treatment facilities.

To see the amount of water (MGD) processed within each county by treatment type, run the cell below.

In [ ]:
df = flow.prep_wastewater_data()
df.head()

### Wastewater Treatment Facility Discharge

Wastewater treatment plant discharge location is provided in the EPA data directly. These values are binned for simplification into the following categories:

* 'outfall to surface waters': 'ww_sd',
* 'ocean discharge': 'ww_od',
* 'deep well': 'ww_gd',
* "reuse: industrial": 'ww_in',
* 'evaporation': 'ww_evap',
* 'spray irrigation': 'ww_ir',
* 'overland flow no discharge': 'ww_ww',
* 'overland flow with discharge': 'ww_sd',
* 'discharge to another facility': 'ww_ww',
* 'combined sewer overflow (cso) discharge': 'ww_sd',
* 'other': 'ww_sd',
* 'discharge to groundwater': 'ww_gd',
* 'no discharge, unknown': 'ww_ww',
* 'reuse: irrigation': 'ww_ir',
* 'reuse: other non-potable': 'ww_sd',
* 'reuse: indirect potable': 'ww_sd',
* 'reuse: potable': 'ww_ps',
* 'reuse: groundwater recharge': 'ww_gd'

In [ ]:
df = flow.prep_wastewater_data()
df = df[['FIPS','ww_evap','ww_gd','ww_in','ww_ir','ww_od','ww_ps','ww_sd','ww_ww']]
df.head()

## Energy in Wastewater Treatment

Wastewater treatment facilities have various levels of treatment from primary to advanced. Each treatment level has an increasing level of energy intensity to process the water. Intensity value estimates for wastwater treatment were used from Pabi et al. [17].

# Agriculture

## Water in Agriculture

## Energy in Agriculture

Energy use in agriculture consists of groundwater pumping, surface water pumping, and interbasin transers. USDA's Farm and Ranch Irrigation Survey (FRIS) provides state-by-state data on irrigation groundwater depth and average irrigation pressurization levels for irrigation within a state, enabling the calculation of pump electricity consumption for both groundwater and surface water pumping [13]. The 2013 survey is the closest year available to 2015 values. It is assumed that values do not vary significantly between the two years.

The methodology for calculating groundwater and surface water pumping energy is described in Tidwell et al [14]. However, where that publication uses well depth to water to calculate total differential height, the total well depth is used here as a way to offset some of the losses due to friction that would occur in the piping, as described in Lawrence Berkeley National Laboratory (LBNL) Home Energy Saver & Score: Engineering Documentation [15].

Pump efficiency is assumed to be the average (46.5%) of the range (34-59%) listed in [14]. State-level intensity rates are calculated here and applied to the county level water in agriculture values. The general methodology implemented here to determine total energy in agriculture is adapted from [5], not including the additions previously mentioned.

### Groundwater & Surface Water Pumping

### Interbasin Transfers

# Electricity Generation

## Energy in Electricity Generation

US Energy Information Administration (EIA) 923 data for 2015 is used to determine electricity generation by generator type within each interconnect [12]. The datafile contains information on generation at the plant-level. EIA additionally provides data on which North American Electric Reliability Corporation (NERC) balancing region each plant belongs to. By mapping each NERC region to its associated interconnect, generation within each interconnect can be determined directly.

From EIA, the AER Fuel Type Code is used to determine generator type. EIA provides generator type information at a high level of detail. For this reason different generator types are grouped for simplicity using a generator key (GenType923Key.xlsx). For example, different types of natural gas generators are all grouped into "Natural Gas" as a type.

Energy production is provided as "Net Generation (Megawatthours)" in the EIA dataset, "Total Fuel Consumption (MMBTU)" is the MMBTU equivalent of fuel going to each plant for electricity generation. The difference between these two values indicates the amount of energy generation that does not go to energy losses.

## will be filled in with calculate function

## Water in Electricity Generation

# End Use Sectors

## Residential Sector

### Water in Residential Sector

### Energy in Residential Sector

#### Energy (electricity) Demand in Residential Sector

##### weighted by population. <b> consider splitting out by usage if data is available

In [ ]:
df = flow.prep_electricity_demand_data()
df

#### Energy (fuel) Demand in Residential Sector

## Commercial Sector

### Water in Commercial Sector

### Energy in Commercial Sector

## Industrial Sector

### Water in Industrial Sector

### Energy in Industrial Sector

## Transportation Sector

### Energy in Energy Sector

# Energy Production

## Biomass

### Energy in Biomass Production (Ethanol)

To find the ethanol production per county, this calculation takes 2015 eia data on ethanol plant capacity which has added locational data by each plant and combines with state level biomass (ethanol) production from prep_state_fuel_production_data() to split out state total by county. Returns a dataframe of ethanol production (bbtu) by county FIPS for each county in the US for 2015.

ethanol plant production filler for wyoming (in state total but not location data) ,Torrington, WY100%, https://neo.ne.gov/programs/stats/122/2015/122_201512.htm

#### <b> <ul> <font color='red'> Consider looking for estimates for electricity use in biomass production to split out from industrial

### Water in Biomass Production (Ethanol)

#### Water used in irrigation of corn used for ethanol

Data on water used in the irrigation of corn ultimately used for ethanol production does not exist explicitly at any level of granularity and must be calculated out from larger state-level corn growth irrigation intensity estimates.

The percent of corn grown for ethanol in the US was used from US DOE Alternative Fuels Data Center [21]. Total corn production for ethanol (5.22 billion bushels) was divided by total corn producton (13.6 billion bushels) for 2015 and calculated to be 38.406%. Data on a state or county-level does not exist, therefore, this percentage is applied to all states with corn production. A state-level fraction of corn used for ethanol production could be calculated from the amount of ethanol produced in a state and applying an assumption of corn required per unit of ethanol, however, not all corn used in ethanol is produced in the state that uses it which lead to states such as CA with an ethanol fraction of corn growth of nearly 900% of their total corn production. Given that we are unable to determine which states the imported corn for ethanol would be coming from, the decision was made to apply the US total fraction to all states.

Total water used in the production of corn for ethanol is therefore the assumed 38.406% multiplied by the product of the total amount of irrigated acres of corn (AC_Irr_Corn) and the calculated irrigation intensity for that state (total acres applied to all crops / total acres irrigated, all crops) provided from USDA Census of Agriculture Irrigation and Water Management Survey [22], Table 35 and Table 4, respectively. The irrigation intensity for corn is calculated from the average irrigation intensity for the state.

To split the total MGY applied between fresh surface and groundwater, USDA FRIS Census of Agriculture data on the fraction of surface water (surface water + off farm water) from total water supply (including groundwater) in corn production is used from Table 37 of [22]. Off-farm sources are assumed to be surface water.

Some states that grow corn do not provide data on the water source breakdown (surface vs. ground) for their irrigation. The surface to groundwater ratio for all irrigation in a state is substituted for these states. These values are calculated from the USGS 2015 water dataset.

In [ ]:
df = flow.prep_county_water_corn_biomass_data()

#### Water used in the production of ethanol (fermentation)

#### <b> <ul> <font color='red'> Consider looking for estimates for water used in the fermentation process of ethanol production, not just corn growth.

## Petroleum

### Energy in Petroleum Production (Conventional and Unconventional Oil Drilling)

 crude oil production by county fillers
Idaho: https://ogcc.idaho.gov/wp-content/uploads/sites/3/2017/06/2016-10-6-state-of-idaho-releases-oil-gas-production-data.pdf
100% of Idaho petroleum and natural gas is from Payette County

Alaska,
https://www.commerce.alaska.gov/web/aogcc/Data.aspx
almost all oil is produced along the north slope. Calculated from oil production download. The rest is on the kenai peninsula (cook inlet basin)

Maryland
https://www.eia.gov/state/analysis.php?sid=MD#34
assumed 50/50 split between Garret and Allegany Counties

Nevada
https://gisweb.unr.edu/OilGas/
Assumed to be in nye county with producing oil wells (only 4 btu)

Oregon
https://www.oregongeology.org/mlrr/oilgas-report.htm
Only one county (columbia county)

### Water in Petroleum Production (Conventional and Unconventional Oil Drilling)

## Natural Gas

### Energy in Natural Gas Production (Unconventional Drilling)

### Water in Natural Gas Production (Unconventional Drilling)

## Coal

### Energy in Coal Production (Coal Mining)

### Water in Coal Production (Coal Mining)

# Intensities & Percentages

#### Conveyance Losses as a Percent of Water Withdrawn for Irrigation
Conveyance loss fraction for irrigation is calculated from the USGS 1995 water use dataset. It is calculated as the total conveyance losses (MGD) divided by the total water withdrawn (MGD) for irrigation. Looking at the fractions at the county level shows that a small subset of counties have unreliable values where conveyance losses are far greater than the amount withdrawn for irrigation. As a result, the ability to cap the loss fraction at a specified amount it provided and replace it with the average of the specified region. The default loss cap fraction is .90.

In [ ]:
df = flow.prep_water_use_1995()
df = flow.calc_conveyance_loss_frac(df)
df.head()

##### Change in loss cap

In [ ]:
df = flow.prep_water_use_1995()
df = flow.calc_conveyance_loss_frac(df, loss_cap_amt=.20)
df.head()

In [ ]:
help(flow.plot_bar)

In [ ]:
flow.plot_bar(df, "State","IR_CLoss_Frac", "State", y_axis_title = "Irrigation Conveyance Loss Fraction", x_axis_title="State")

#### Water Use Intensity for Hydroelectric Generation

In [ ]:
df = flow.calc_hydroelectric_water_intensity(intensity_cap=True, intensity_cap_amt=165, region_avg=False)

df.head()

In [ ]:
flow.plot_bar(df, "State","HY_IF", "State")